In [ ]:
import numpy as np
from dataclasses import dataclass

'''
To Do

- add lactate fermentation
- refine glucose metabolism pathway, add "burning"
- have insulin and glucagon modulate metabolism of glucose, FA, and AA
- refine oxidative phosphorylation pathway

- Ectopic fat storage
- inflammation management
- have protein intake as well as growth hormones modulate muscle mass (volume)
- add clearances


'''

n=9

@dataclass
class Parameters:
    insulin_blood_skeletalmuscle: float
    insulin_skeletalmuscle_blood: float

    V_skeletalmuscle: float

    glucagon_blood_skeletalmuscle: float
    glucagon_skeletalmuscle_blood: float

    fattyacids_blood_skeletalmuscle: float
    fattyacids_skeletalmuscle_blood: float

    glucose_blood_skeletalmuscle: float
    glucose_skeletalmuscle_blood: float

    aminoacids_blood_skeletalmuscle: float
    aminoacids_skeletalmuscle_blood: float

    lactate_blood_skeletalmuscle: float
    lactate_skeletalmuscle_blood: float

    growthhormone_blood_skeletalmuscle: float
    growthhormone_skeletalmuscle_blood: float

    V_blood: float

    isExercising: bool

    glucose_to_g6p: float
    g6p_to_glucose: float

    g6p_to_glycogen: float
    glycogen_to_g6p: float
    

p = Parameters(
    insulin_blood_skeletalmuscle = 1,
    insulin_skeletalmuscle_blood=1,
    V_skeletalmuscle=1,
    glucagon_blood_skeletalmuscle=1,
    glucagon_skeletalmuscle_blood=1,
    fattyacids_blood_skeletalmuscle=1,
    fattyacids_skeletalmuscle_blood=1,
    glucose_blood_skeletalmuscle=1,
    glucose_skeletalmuscle_blood=1,
    aminoacids_blood_skeletalmuscle=1,
    aminoacids_skeletalmuscle_blood=1,
    lactate_blood_skeletalmuscle=1,
    lactate_skeletalmuscle_blood=1,
    growthhormone_blood_skeletalmuscle=1,
    growthhormone_skeletalmuscle_blood=1,
    V_blood=1,
    isExercizing=False,
    glucose_to_g6p = 1,
    g6p_to_glucose = 1,
    g6p_to_glycogen = 1,
    glycogen_to_g6p = 1

)

def skeletalmuscle(t, y, p):
    dydt = np.zeros(n)
    
    dinsulin = insulin(t, y, p)
    dglucagon = glucagon(t, y, p)
    dfattyacids = fattyacids(t, y, p)
    dglucose = glucose(t, y, p)
    daminoacids = aminoacids(t, y, p)
    dlactate = lactate(t, y, p)
    dgrowthhormone = growthhormone(t, y, p)
    dg6p = g6p(t,y,p)
    dglycogen = glycogen(t,y,p)
    
    dydt = dinsulin + dglucagon + dfattyacids + dglucose + daminoacids + dlactate + dgrowthhormone + dg6p + dglycogen
    return dydt

def insulin(t, y, p):
    dydt = np.zeros(n)
    dydt[0] = (p.insulin_blood_skeletalmuscle * y[0] *p.V_blood - p.insulin_skeletalmuscle_blood * y[0] * p.V_skeletalmuscle) / p.V_skeletalmuscle
    return dydt

def glucagon(t, y, p):
    dydt = np.zeros(n)
    dydt[1] = (p.glucagon_blood_skeletalmuscle * y[1] * p.V_blood - p.glucagon_skeletalmuscle_blood * y[1] * p.V_skeletalmuscle) / p.V_skeletalmuscle
    return dydt

def fattyacids(t, y, p):
    dydt = np.zeros(n)
    dydt[2] = (p.fattyacids_blood_skeletalmuscle * y[2] * p.V_blood - p.fattyacids_skeletalmuscle_blood * y[2] * p.V_skeletalmuscle) / p.V_skeletalmuscle
    return dydt

def glucose(t, y, p):
    dydt = np.zeros(n)
    dydt[3] = (
        (p.glucose_blood_skeletalmuscle * y[3] * p.V_blood - p.glucose_skeletalmuscle_blood * y[3] * p.V_skeletalmuscle) / p.V_skeletalmuscle
        -p.glucose_to_g6p * y[3] + p.g6p_to_glucose *y[7]
    )
    return dydt

def aminoacids(t, y, p):
    dydt = np.zeros(n)
    dydt[4] = (p.aminoacids_blood_skeletalmuscle * y[4] * p.V_blood - p.aminoacids_skeletalmuscle_blood * y[4] * p.V_skeletalmuscle) / p.V_skeletalmuscle
    return dydt

def lactate(t, y, p):
    dydt = np.zeros(n)
    dydt[5] = (p.lactate_blood_skeletalmuscle * y[5] * p.V_blood - p.lactate_skeletalmuscle_blood * y[5] * p.V_skeletalmuscle) / p.V_skeletalmuscle
    return dydt

def growthhormone(t, y, p):
    dydt = np.zeros(n)
    dydt[6] = (p.growthhormone_blood_skeletalmuscle * y[6] * p.V_blood - p.growthhormone_skeletalmuscle_blood * y[6] * p.V_skeletalmuscle) / p.V_skeletalmuscle
    return dydt

def g6p(t,y,p):
    dydt = np.zeros(n)
    if p.isExercising:
        glu2g6p = p.glucose_to_g6p * 1
        g6p2glu = p.g6p_to_glucose * 1
        g6p2gly = p.g6p_to_glycogen * 1
        gly2g6p = p.glycogen_to_g6p * 1
    else:
        glu2g6p = p.glucose_to_g6p * 1
        g6p2glu = p.g6p_to_glucose * 1
        g6p2gly = p.g6p_to_glycogen * 1
        gly2g6p = p.glycogen_to_g6p * 1

    dydt[7] = (
        glu2g6p * y[3] - g6p2glu *y[7] - g6p2gly*y[3] +gly2g6p*y[8]
    )

    return dydt

def glycogen(t,y,p):
    dydt = np.zeros(n)
    if p.isExercising:
        g6p2gly = p.g6p_to_glycogen * 1
        gly2g6p = p.glycogen_to_g6p * 1
    elif y[8]>= p.V_skeletalmuscle*0.2:
        g6p2gly = p.g6p_to_glycogen
        gly2g6p = g6p2gly
    else:
        g6p2gly = p.g6p_to_glycogen * 1
        gly2g6p = p.glycogen_to_g6p * 1

    dydt[8] = (
        g6p2gly*y[3] -gly2g6p*y[8]
    )

    return dydt
